# Experiment 3 forecasting of S&P500 and AEX

## Libraries and Drive

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
import os
os.chdir('/content/drive/My Drive/Volatility_forecasting/')

In [ ]:
!pip install neuralforecast ruptures dask[dataframe] scikit-learn
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'

## S&P500

In [ ]:
import pandas as pd
y_actual = pd.read_excel('Data/S&P500/S&P500_840E.xlsx', sheet_name='Sheet1')

y_actual = y_actual.melt(id_vars=['Name'], var_name='Date', value_name='840E')

y_actual['Name'] = y_actual['Name'].str.split(' - ').str[0]
y_actual.columns = ['unique_id', 'ds', '840E']
y_actual = y_actual.rename(columns={'840E': 'y'})
y_actual['ds'] = pd.to_datetime(y_actual['ds'])
y_actual

,unique_id,ds,y
0,AMAZON.COM,1997-05-15,NaN
1,ABBOTT LABORATORIES,1997-05-15,0.2833
2,AES,1997-05-15,0.3158
3,INTERNATIONAL BUS.MCHS.,1997-05-15,0.3466
4,ADVANCED MICRO DEVICES,1997-05-15,0.5886
...,...,...,...
3575319,WEC ENERGY GROUP,2024-08-12,0.1867
3575320,MONSTER BEVERAGE,2024-08-12,0.3751
3575321,SBA COMMS.,2024-08-12,0.3171
3575322,CHARTER COMMS.CL.A,2024-08-12,0.5525


In [ ]:
y_actual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3575324 entries, 0 to 3575323
Data columns (total 3 columns):
 #   Column     Dtype         
---  ------     -----         
 0   unique_id  object        
 1   ds         datetime64[ns]
 2   y          float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 81.8+ MB


In [ ]:
df = pd.read_csv('Data/S&P500/3ProSP500.csv')
df = df.rename(columns={'840E': 'y'})
df['ds'] = pd.to_datetime(df['ds'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008545 entries, 0 to 2008544
Data columns (total 21 columns):
 #   Column     Dtype         
---  ------     -----         
 0   unique_id  object        
 1   ds         datetime64[ns]
 2   y          float64       
 3   MA(2,9)    int64         
 4   MA(3,9)    int64         
 5   MA(1,12)   int64         
 6   MA(2,12)   int64         
 7   MA(3,12)   int64         
 8   MOM(9)     int64         
 9   MOM(12)    int64         
 10  RSI(7)     int64         
 11  RSI(14)    int64         
 12  EMA(3,9)   int64         
 13  EMA(5,9)   int64         
 14  EMA(5,12)  int64         
 15  DY         float64       
 16  PTBV       float64       
 17  P          float64       
 18  PO         float64       
 19  VO         float64       
 20  PE         float64       
dtypes: datetime64[ns](1), float64(7), int64(12), object(1)
memory usage: 321.8+ MB


In [ ]:
df = df.sort_values(by=['unique_id', 'ds'])
df

,unique_id,ds,y,"MA(2,9)","MA(3,9)","MA(1,12)","MA(2,12)","MA(3,12)",MOM(9),MOM(12),...,RSI(14),"EMA(3,9)","EMA(5,9)","EMA(5,12)",DY,PTBV,P,PO,VO,PE
0,3M,2004-01-01,0.1520,0,0,0,0,0,0,0,...,0,0,0,0,1.55,6.34,71.0915,72.545513,30974.896916,29.4
1,3M,2004-01-02,0.1649,0,0,0,0,0,0,0,...,0,0,0,0,1.58,6.21,69.7036,71.066400,3100.100000,28.8
2,3M,2004-01-05,0.1571,0,0,0,0,0,0,0,...,0,0,0,0,1.55,6.33,71.0329,69.879200,3570.500000,29.3
3,3M,2004-01-06,0.1564,0,0,0,0,0,0,0,...,0,0,0,0,1.56,6.31,70.8156,70.648400,3568.200000,29.3
4,3M,2004-01-07,0.1664,0,0,0,0,0,0,0,...,0,0,0,0,1.59,6.20,69.5949,70.330600,5135.400000,28.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008540,ZIMMER BIOMET HDG.,2023-12-25,0.1275,1,1,1,1,1,1,1,...,0,1,1,1,0.80,1.99,120.5600,121.197601,106671.248037,53.4
2008541,ZIMMER BIOMET HDG.,2023-12-26,0.1178,1,1,1,1,1,1,1,...,0,1,1,1,0.79,2.00,121.4200,120.220000,1870.700000,53.8
2008542,ZIMMER BIOMET HDG.,2023-12-27,0.1055,1,1,1,1,1,1,1,...,0,1,1,1,0.79,2.00,121.1300,121.200000,1058.600000,53.7
2008543,ZIMMER BIOMET HDG.,2023-12-28,0.1038,1,1,1,1,1,1,1,...,0,1,1,1,0.79,2.00,121.6300,120.930000,662.200000,53.9


### Predicting

In [ ]:
from neuralforecast.core import NeuralForecast

auto_models = ['AutoTiDE', 'AutoTFT', 'AutoNHITS', 'AutoBiTCN']  # Full model names for paths
models = ['tide', 'tft', 'nhits', 'bitcn']  # Simplified model names for output files
horizons = [5, 10, 20]

base_path = 'Trained Models/{auto_model}/horizon_{horizon}/'
output_path = 'Data/Test/horizon_{horizon}/SP500{model}_model0_1_horizon_{horizon}.csv'

for auto_model, model in zip(auto_models, models):  # Iterate over both lists in parallel
    for horizon in horizons:
        # Generate the model path dynamically
        model_path = base_path.format(auto_model=auto_model, horizon=horizon)

        try:
            # Load the trained model for the specific model and horizon
            nf = NeuralForecast.load(path=model_path)

            # Make predictions using the model
            Y_hat_df = nf.predict(df=df)

            # Merge predictions with actual values
            merged_df = pd.merge(y_actual, Y_hat_df, on=['unique_id', 'ds'], how='inner')

            # Save the merged DataFrame to a CSV file
            merged_df.to_csv(output_path.format(model=model, horizon=horizon), index=False)

            print(f'Successfully processed Model {auto_model} ({model}) for Horizon {horizon}.')
        except FileNotFoundError as e:
            print(f"Error: {e}")
            print(f"Skipping Model {auto_model} ({model}) for Horizon {horizon} due to missing files.")

/usr/local/lib/python3.10/dist-packages/neuralforecast/common/_base_model.py:444: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  content = torch.load(f, **kwargs)
INFO:lightn

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Successfully processed Model AutoTiDE (tide) for Horizon 5.


...

/usr/local/lib/python3.10/dist-packages/neuralforecast/common/_base_model.py:444: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  content = torch.load(f, **kwargs)
INFO:lightn

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Successfully processed Model AutoBiTCN (bitcn) for Horizon 20.


### Merging

In [ ]:
import pandas as pd
import os

class HorizonMerger:
    def __init__(self, base_dir='Data/Test', models=['bitcn', 'tide', 'tft', 'nhits'],
                 model_version='model0_1', horizons=[1], prefixes=[None]):
        """
        Initializes the HorizonMerger with specified parameters.

        Parameters:
        - base_dir (str): Base directory where data is stored.
        - models (list): List of model names to process.
        - model_version (str): Version identifier for the models.
        - horizons (list): List of horizon values to process.
        - prefixes (list): List of prefixes to process, e.g., ['', '8TY'].
        """
        self.base_dir = base_dir
        self.models = models
        self.model_version = model_version
        self.horizons = horizons
        self.prefixes = prefixes  # List of prefixes
        # Mapping from model names to their corresponding column prefixes
        self.model_column_mapping = {
            'bitcn': 'AutoBiTCN',
            'tide': 'AutoTiDE',
            'tft': 'AutoTFT',
            'nhits': 'AutoNHITS'
        }

    def merge_models_for_horizon_and_prefix(self, horizon, prefix):
        """
        Merges CSV files for a specific horizon and prefix, then saves the full and subset data.

        Parameters:
        - horizon (int): The horizon value to process.
        - prefix (str): The prefix to add to the filename.
        """
        # List to store individual DataFrames
        df_list = []

        # Path to the current horizon's directory
        horizon_dir = os.path.join(self.base_dir, f'horizon_{horizon}')

        # Ensure the horizon directory exists
        if not os.path.isdir(horizon_dir):
            print(f"⚠️ Horizon directory does not exist: {horizon_dir}. Creating it...")
            os.makedirs(horizon_dir, exist_ok=True)

        # Read each model's CSV for the current horizon
        for model in self.models:
            # Update file name to include the prefix
            file_name = f'{prefix or ""}{model}_{self.model_version}_horizon_{horizon}.csv'
            file_path = os.path.join(horizon_dir, file_name)
            if not os.path.exists(file_path):
                print(f"❌ File not found: {file_path}")
                return  # Skip processing this horizon if any file is missing
            df = pd.read_csv(file_path)
            df_list.append(df)
            print(f"✅ Loaded {file_path}")

        # Merge all DataFrames on the specified keys
        print(f"🔗 Merging data for horizon {horizon} with prefix '{prefix}'...")
        merged_df = df_list[0]
        for df in df_list[1:]:
            merged_df = pd.merge(merged_df, df, on=['unique_id', 'ds', 'y'], how='inner')
        print("✅ Merging completed.")

        # Define the output filename and path for the full merged data
        full_output_filename = f'{prefix or ""}{"_".join(self.models)}_{self.model_version}_full_horizon_{horizon}.csv'
        full_output_path = os.path.join(horizon_dir, full_output_filename)

        # Save the merged DataFrame
        merged_df.to_csv(full_output_path, index=False)
        print(f"💾 Saved merged data to {full_output_path}")

        # Prepare subset columns based on the mapping
        subset_columns = ['unique_id', 'ds', 'y']
        for model in self.models:
            col_prefix = self.model_column_mapping.get(model)
            if col_prefix:
                subset_columns.extend([col_prefix, f'{col_prefix}1'])
            else:
                print(f"⚠️ No column mapping found for model '{model}'. Skipping its subset columns.")

        # Check for missing columns and handle them
        missing_cols = [col for col in subset_columns if col not in merged_df.columns]
        if missing_cols:
            print(f"⚠️ Missing columns for subset: {missing_cols}")
            for col in missing_cols:
                merged_df[col] = pd.NA

        # Create the subset DataFrame
        df_subset = merged_df[subset_columns].copy()

        # Define the output filename and path for the subset data
        subset_output_filename = f'{prefix or ""}{"_".join(self.models)}_{self.model_version}_sub_horizon_{horizon}.csv'
        subset_output_path = os.path.join(horizon_dir, subset_output_filename)

        # Save the subset DataFrame
        df_subset.to_csv(subset_output_path, index=False)
        print(f"💾 Saved subset data to {subset_output_path}\n")

    def process_all_horizons_and_prefixes(self):
        """
        Processes all specified horizons and prefixes by merging and saving their respective data.
        """
        for prefix in self.prefixes:
            for horizon in self.horizons:
                print(f"🚀 Processing horizon {horizon} with prefix '{prefix}'...")
                self.merge_models_for_horizon_and_prefix(horizon, prefix)
                print(f"✅ Finished processing horizon {horizon} for prefix '{prefix}'\n")

if __name__ == "__main__":
    # Initialize the HorizonMerger with horizons and prefixes
    merger = HorizonMerger(
        horizons=[5, 10, 20],
        prefixes=['SP500']
    )

    # Start processing all specified horizons and prefixes
    merger.process_all_horizons_and_prefixes()
    print("✅ Completed all processing")

🚀 Processing horizon 5 with prefix 'SP500'...
✅ Loaded Data/Test/horizon_5/SP500bitcn_model0_1_horizon_5.csv
✅ Loaded Data/Test/horizon_5/SP500tide_model0_1_horizon_5.csv
✅ Loaded Data/Test/horizon_5/SP500tft_model0_1_horizon_5.csv
✅ Loaded Data/Test/horizon_5/SP500nhits_model0_1_horizon_5.csv
🔗 Merging data for horizon 5 with prefix 'SP500'...
✅ Merging completed.
💾 Saved merged data to Data/Test/horizon_5/SP500bitcn_tide_tft_nhits_model0_1_full_horizon_5.csv
💾 Saved subset data to Data/Test/horizon_5/SP500bitcn_tide_tft_nhits_model0_1_sub_horizon_5.csv

✅ Finished processing horizon 5 for prefix 'SP500'

🚀 Processing horizon 10 with prefix 'SP500'...
✅ Loaded Data/Test/horizon_10/SP500bitcn_model0_1_horizon_10.csv
✅ Loaded Data/Test/horizon_10/SP500tide_model0_1_horizon_10.csv
✅ Loaded Data/Test/horizon_10/SP500tft_model0_1_horizon_10.csv
✅ Loaded Data/Test/horizon_10/SP500nhits_model0_1_horizon_10.csv
🔗 Merging data for horizon 10 with prefix 'SP500'...
✅ Merging completed.
💾 Saved 

### Evaluation

Following the evaluation metrics as well as MCS and DM test from Suoto and Moradi 2024

https://www.emerald.com/insight/content/doi/10.1108/cfri-01-2024-0032/full/html

https://github.com/hugogobato/Can-Transformers-Transform-Financial-Forecasting-

In [ ]:
import os
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import t
from arch.bootstrap import MCS

# Define the horizons and prefixes
horizons = [5, 10, 20]
prefixes = ['SP500']

# Loop over each prefix and horizon
for prefix in prefixes:
    for horizon in horizons:
        print(f"Processing Prefix: '{prefix}', Horizon: {horizon}")

        # Define file paths based on prefix and horizon
        prefix_str = f"{prefix}_" if prefix else ''
        file_name = f"{prefix}bitcn_tide_tft_nhits_model0_1_sub_horizon_{horizon}.csv"
        data_path = f"Data/Test/horizon_{horizon}/{file_name}"
        output_dir = f"Data/Evaluation/horizon_{horizon}"
        os.makedirs(output_dir, exist_ok=True)

        # Read the CSV file
        df = pd.read_csv(data_path)
        df = df.sort_values(by=['unique_id', 'ds']).drop_duplicates(subset=['ds', 'unique_id'])
        df['ds'] = pd.to_datetime(df['ds'])

        # Extract model columns
        model_columns = ['AutoTiDE', 'AutoTiDE1', 'AutoTFT', 'AutoTFT1',
                         'AutoBiTCN', 'AutoBiTCN1', 'AutoNHITS', 'AutoNHITS1']

        # Create a dictionary to hold pivoted DataFrames for each model
        model_dfs = {}
        for model in model_columns:
            model_df = df.pivot(index='ds', columns='unique_id', values=model)
            model_dfs[model] = model_df

        # Pivot actual values
        Actuals = df.pivot(index='ds', columns='unique_id', values='y')

        # Calculate error metrics for each model
        metrics = ['RMSE', 'MAE', 'MAPE', 'QLIKE']
        error_metrics = {metric: {} for metric in metrics}

        for model_name, predictions in model_dfs.items():
            # Align the predictions and actuals
            predictions, actuals = predictions.align(Actuals, join='inner', axis=0)
            errors = predictions - actuals

            # RMSE Calculation
            mse = (errors ** 2).mean().mean()
            error_metrics['RMSE'][model_name] = np.sqrt(mse)

            # MAE Calculation
            mae = errors.abs().mean().mean()
            error_metrics['MAE'][model_name] = mae

            # MAPE Calculation
            with np.errstate(divide='ignore', invalid='ignore'):
                mape = (errors.abs() / actuals).replace([np.inf, -np.inf], np.nan).mean().mean()
            error_metrics['MAPE'][model_name] = mape

            # QLIKE Calculation
            with np.errstate(divide='ignore', invalid='ignore'):
                ratio = actuals / predictions
                qlike = (ratio - np.log(ratio) - 1).replace([np.inf, -np.inf], np.nan).mean().mean()
            error_metrics['QLIKE'][model_name] = qlike

        # Create a DataFrame with error metrics
        error_metrics_df = pd.DataFrame(error_metrics)
        metrics_file = f"{output_dir}/{prefix_str}metrics_horizon_{horizon}.csv"
        error_metrics_df.to_csv(metrics_file)

        # Function to calculate losses for MCS
        def calculate_losses(metric, model_dfs, actuals):
            residuals_dict = {}
            for model_name, predictions in model_dfs.items():
                # Align predictions and actuals
                predictions_aligned, actuals_aligned = predictions.align(actuals, join='inner', axis=0)
                if metric == 'RMSE':
                    residuals = (predictions_aligned - actuals_aligned) ** 2
                elif metric == 'MAE':
                    residuals = (predictions_aligned - actuals_aligned).abs()
                elif metric == 'MAPE':
                    with np.errstate(divide='ignore', invalid='ignore'):
                        residuals = (predictions_aligned - actuals_aligned).abs() / actuals_aligned
                elif metric == 'QLIKE':
                    with np.errstate(divide='ignore', invalid='ignore'):
                        ratio = actuals_aligned / predictions_aligned
                        residuals = ratio - np.log(ratio) - 1
                else:
                    raise ValueError("Unsupported metric")

                residuals = residuals.replace([np.inf, -np.inf], np.nan)
                residuals_dict[model_name] = residuals.mean(axis=1)

            return pd.DataFrame(residuals_dict).dropna()

        # Run MCS procedure for each metric
        pvalues_list = []

        for metric in metrics:
            losses = calculate_losses(metric, model_dfs, Actuals)
            mcs = MCS(losses, size=0.05, method="R", block_size=1000)
            mcs.compute()
            pvalues = mcs.pvalues.reset_index().rename(columns={'Pvalue': f'Pvalue_{metric}'})
            pvalues_list.append(pvalues)

        # Merge p-values and save
        merged_pvalues = pvalues_list[0]
        for pvalues in pvalues_list[1:]:
            merged_pvalues = merged_pvalues.merge(pvalues, on='Model name', how='outer')

        mcs_file = f"{output_dir}/{prefix_str}MCS_horizon_{horizon}.csv"
        merged_pvalues.to_csv(mcs_file, index=False)

        # Function for Diebold-Mariano test
        def dm_test(actual, pred1, pred2, h=1, crit="RMSE"):
            actual, pred1, pred2 = map(np.array, [actual, pred1, pred2])
            T = len(actual)

            # Ensure predictions and actuals are of the same length
            if len(actual) != len(pred1) or len(actual) != len(pred2):
                return np.nan

            # Remove NaN values
            mask = ~np.isnan(actual) & ~np.isnan(pred1) & ~np.isnan(pred2)
            actual = actual[mask]
            pred1 = pred1[mask]
            pred2 = pred2[mask]
            T = len(actual)
            if T == 0:
                return np.nan

            if crit == "RMSE":
                loss_diff = (actual - pred1) ** 2 - (actual - pred2) ** 2
            elif crit == "MAE":
                loss_diff = np.abs(actual - pred1) - np.abs(actual - pred2)
            elif crit == "MAPE":
                with np.errstate(divide='ignore', invalid='ignore'):
                    loss_diff = (np.abs(actual - pred1) / actual) - (np.abs(actual - pred2) / actual)
            elif crit == "QLIKE":
                with np.errstate(divide='ignore', invalid='ignore'):
                    loss1 = actual / np.abs(pred1) - np.log(actual / np.abs(pred1)) - 1
                    loss2 = actual / np.abs(pred2) - np.log(actual / np.abs(pred2)) - 1
                    loss_diff = loss1 - loss2
            else:
                raise ValueError("Unsupported criterion")

            loss_diff = loss_diff[~np.isnan(loss_diff)]
            if len(loss_diff) == 0:
                return np.nan

            d_mean = np.mean(loss_diff)
            gamma = [np.correlate(loss_diff - d_mean, loss_diff - d_mean, 'full')[len(loss_diff)-1:] / len(loss_diff)][0]
            V_d = gamma[0] + 2 * sum(gamma[1:h])
            DM_stat = d_mean / np.sqrt(V_d / len(loss_diff))
            adj = ((len(loss_diff) + 1 - 2 * h + h * (h - 1) / len(loss_diff)) / len(loss_diff)) ** 0.5
            return DM_stat * adj

        # Run DM tests and save results
        models = model_columns
        stocks = Actuals.columns

        for metric in metrics:
            better_count_matrix = pd.DataFrame(0, index=models, columns=models)

            for model_a, model_b in combinations(models, 2):
                counts = {'a_better': 0, 'b_better': 0}
                for stock in stocks:
                    actual = Actuals[stock]
                    pred1 = model_dfs[model_a][stock]
                    pred2 = model_dfs[model_b][stock]
                    combined = pd.concat([actual, pred1, pred2], axis=1).dropna()

                    if combined.empty:
                        continue

                    dm_stat = dm_test(combined.iloc[:, 0], combined.iloc[:, 1], combined.iloc[:, 2], h=1, crit=metric)
                    if np.isnan(dm_stat):
                        continue
                    if dm_stat > 1.96:
                        counts['a_better'] += 1
                    elif dm_stat < -1.96:
                        counts['b_better'] += 1

                better_count_matrix.loc[model_a, model_b] = counts['a_better']
                better_count_matrix.loc[model_b, model_a] = counts['b_better']

            # Summarize results
            better_count_matrix['Outperform Count'] = better_count_matrix.sum(axis=1)
            better_count_matrix.loc['Outperformed Count'] = better_count_matrix.sum(axis=0)
            better_count_matrix.loc['Outperformed Count', 'Outperform Count'] = np.nan
            dm_file = f"{output_dir}/{prefix_str}DM_{metric}_horizon_{horizon}.csv"
            better_count_matrix.to_csv(dm_file)

        print(f"Completed processing for Prefix: '{prefix}', Horizon: {horizon}\n")

Processing Prefix: 'SP500', Horizon: 5
Completed processing for Prefix: 'SP500', Horizon: 5

Processing Prefix: 'SP500', Horizon: 10
Completed processing for Prefix: 'SP500', Horizon: 10

Processing Prefix: 'SP500', Horizon: 20
Completed processing for Prefix: 'SP500', Horizon: 20



#### Metrics

In [ ]:
import pandas as pd

# Load the data for each horizon and merge by metric
horizons = [5, 10, 20]
metrics = ['RMSE', 'MAE', 'MAPE', 'QLIKE']
prefixes = ['SP500_']

metric_data = {metric: [] for metric in metrics}

# Process each horizon, prefix, and add to the corresponding metric dictionary
for prefix in prefixes:
    for horizon in horizons:
        # Adjust file path with prefix
        file_path = f'Data/Evaluation/horizon_{horizon}/{prefix}metrics_horizon_{horizon}.csv'
        try:
            df = pd.read_csv(file_path)
            df = df.rename(columns={'Unnamed: 0': 'model'})
            df['model'] = df['model'].str.replace('Auto', '', regex=False)
            for metric in metrics:
                metric_df = df[['model', metric]].copy()
                metric_df = metric_df.rename(columns={metric: f'{metric}_{prefix}horizon_{horizon}'})
                metric_data[metric].append(metric_df)
        except FileNotFoundError:
            print(f"File not found: {file_path}. Skipping this file.")

# Create the output directory if it doesn't exist
output_dir = 'Data/Evaluation/Final'
os.makedirs(output_dir, exist_ok=True)

# Export each metric DataFrame to an Excel file
with pd.ExcelWriter(f'{output_dir}/SP500_metrics_across_horizons_and_prefixes.xlsx') as writer:
    for metric in metrics:
        # Concatenate the metric dataframes for each metric across horizons
        if metric_data[metric]:  # Check if data exists for the metric
            result_df = pd.concat(metric_data[metric], axis=1)
            result_df = result_df.loc[:,~result_df.columns.duplicated()]  # remove duplicate 'model' columns
            # Write each metric to a separate sheet in the Excel file
            result_df.to_excel(writer, sheet_name=metric, index=False)

"Excel file with metrics across horizons and prefixes has been created in 'Data/Evaluation/Final/'."

"Excel file with metrics across horizons and prefixes has been created in 'Data/Evaluation/Final/'."

#### DM

In [ ]:
import pandas as pd
import os

# Initialize dictionaries to store data for each metric and horizon
metrics = ['RMSE', 'MAE', 'MAPE', 'QLIKE']
prefixes = ['SP500_']
horizons = [5, 10, 20]
metric_data = {metric: [] for metric in metrics}

# Process each metric and compile data across all horizons and prefixes
for metric in metrics:
    for prefix in prefixes:
        for horizon in horizons:
            # Define file path and check if it exists
            file_path = f"Data/Evaluation/horizon_{horizon}/{prefix}DM_{metric}_horizon_{horizon}.csv"
            if os.path.exists(file_path):
                # Read the data, rename columns, and filter as specified
                df = pd.read_csv(file_path)
                df = df.rename(columns={'Unnamed: 0': 'model', 'Outperform Count': f'{metric}_{prefix.strip("_")}_h{horizon}_Outperform'})
                df = df[df['model'] != 'Outperformed Count']
                df = df[['model', f'{metric}_{prefix.strip("_")}_h{horizon}_Outperform']]
                df['model'] = df['model'].str.replace('Auto', '', regex=False)
                metric_data[metric].append(df)

# Save data for each metric into an Excel file with each horizon's data in one sheet per metric
with pd.ExcelWriter('Data/Evaluation/Final/SP500_DM_across_horizons_and_prefixes.xlsx') as writer:
    for metric, data_frames in metric_data.items():
        # Concatenate data frames for the specific metric along columns by 'model' as the index
        result_df = pd.concat(data_frames, axis=1).loc[:,~pd.concat(data_frames, axis=1).columns.duplicated()]
        result_df.to_excel(writer, sheet_name=metric, index=False)

'Excel file created with summary of all metrics.'

'Excel file created with summary of all metrics.'

## AEX

In [ ]:
import pandas as pd
y_actual = pd.read_excel('Data/AEX/AEX_840E.xlsx', sheet_name='Sheet1')

y_actual = y_actual.melt(id_vars=['Name'], var_name='Date', value_name='840E')

y_actual['Name'] = y_actual['Name'].str.split(' - ').str[0]
y_actual.columns = ['unique_id', 'ds', '840E']
y_actual = y_actual.rename(columns={'840E': 'y'})
y_actual['ds'] = pd.to_datetime(y_actual['ds'])
y_actual

,unique_id,ds,y
0,ABN AMRO BANK,2015-11-19,NaN
1,ADYEN,2015-11-19,NaN
2,AEGON,2015-11-19,0.4380
3,KONINKLIJKE AHOLD DELHAIZE,2015-11-19,0.2208
4,AKZO NOBEL,2015-11-19,0.2576
...,...,...,...
57870,RELX (AMS),2024-10-02,0.1602
57871,SHELL,2024-10-02,0.2233
57872,UNIVERSAL MUSIC GROUP,2024-10-02,0.1938
57873,UNILEVER (UK) (AMS),2024-10-02,0.1021


In [ ]:
y_actual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57875 entries, 0 to 57874
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   unique_id  57875 non-null  object        
 1   ds         57875 non-null  datetime64[ns]
 2   y          50559 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.3+ MB


In [ ]:
df = pd.read_csv('Data/AEX/3ProAEX.csv')
df = df.rename(columns={'840E': 'y'})
df['ds'] = pd.to_datetime(df['ds'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42246 entries, 0 to 42245
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   unique_id  42246 non-null  object        
 1   ds         42246 non-null  datetime64[ns]
 2   y          42246 non-null  float64       
 3   MA(2,9)    42246 non-null  int64         
 4   MA(3,9)    42246 non-null  int64         
 5   MA(1,12)   42246 non-null  int64         
 6   MA(2,12)   42246 non-null  int64         
 7   MA(3,12)   42246 non-null  int64         
 8   MOM(9)     42246 non-null  int64         
 9   MOM(12)    42246 non-null  int64         
 10  RSI(7)     42246 non-null  int64         
 11  RSI(14)    42246 non-null  int64         
 12  EMA(3,9)   42246 non-null  int64         
 13  EMA(5,9)   42246 non-null  int64         
 14  EMA(5,12)  42246 non-null  int64         
 15  DY         42246 non-null  float64       
 16  PTBV       42246 non-null  float64      

In [ ]:
df = df.sort_values(by=['unique_id', 'ds'])
df

,unique_id,ds,y,"MA(2,9)","MA(3,9)","MA(1,12)","MA(2,12)","MA(3,12)",MOM(9),MOM(12),...,RSI(14),"EMA(3,9)","EMA(5,9)","EMA(5,12)",DY,PTBV,P,PO,VO,PE
0,ABN AMRO BANK,2016-01-01,0.2049,0,0,0,0,0,0,0,...,0,0,0,0,0.00,1.08,20.670,21.130379,0.000000,9.2
1,ABN AMRO BANK,2016-01-04,0.2030,0,0,0,0,0,0,0,...,0,0,0,0,0.00,1.08,20.670,20.600000,898.300000,9.2
2,ABN AMRO BANK,2016-01-05,0.2025,0,0,0,0,0,0,0,...,0,0,0,0,0.00,1.08,20.630,20.870000,827.700000,9.1
3,ABN AMRO BANK,2016-01-06,0.1959,0,0,0,0,0,0,0,...,0,0,0,0,0.00,1.05,20.130,20.650000,757.700000,8.9
4,ABN AMRO BANK,2016-01-07,0.2068,0,0,0,0,0,0,0,...,0,0,0,0,0.00,1.03,19.695,19.965000,1498.800000,8.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42241,WOLTERS KLUWER,2023-12-25,0.1267,0,0,0,0,0,0,0,...,1,0,0,0,1.47,17.77,129.200,129.615123,8742.962801,30.7
42242,WOLTERS KLUWER,2023-12-26,0.1253,0,0,0,0,0,0,0,...,1,0,0,0,1.47,17.77,129.200,129.384568,0.000000,30.7
42243,WOLTERS KLUWER,2023-12-27,0.1285,0,0,0,0,0,0,0,...,1,0,0,0,1.48,17.62,128.150,129.150000,415.600000,30.5
42244,WOLTERS KLUWER,2023-12-28,0.1284,0,0,0,0,0,0,0,...,1,0,0,0,1.48,17.61,128.050,128.450000,327.600000,30.5


#### Predicting

In [ ]:
from neuralforecast.core import NeuralForecast

auto_models = ['AutoTiDE', 'AutoTFT', 'AutoNHITS', 'AutoBiTCN']  # Full model names for paths
models = ['tide', 'tft', 'nhits', 'bitcn']  # Simplified model names for output files
horizons = [5, 10, 20]

base_path = 'Trained Models/{auto_model}/horizon_{horizon}/'
output_path = 'Data/Test/horizon_{horizon}/AEX{model}_model0_1_horizon_{horizon}.csv'

for auto_model, model in zip(auto_models, models):  # Iterate over both lists in parallel
    for horizon in horizons:
        # Generate the model path dynamically
        model_path = base_path.format(auto_model=auto_model, horizon=horizon)

        try:
            # Load the trained model for the specific model and horizon
            nf = NeuralForecast.load(path=model_path)

            # Make predictions using the model
            Y_hat_df = nf.predict(df=df)

            # Merge predictions with actual values
            merged_df = pd.merge(y_actual, Y_hat_df, on=['unique_id', 'ds'], how='inner')

            # Save the merged DataFrame to a CSV file
            merged_df.to_csv(output_path.format(model=model, horizon=horizon), index=False)

            print(f'Successfully processed Model {auto_model} ({model}) for Horizon {horizon}.')
        except FileNotFoundError as e:
            print(f"Error: {e}")
            print(f"Skipping Model {auto_model} ({model}) for Horizon {horizon} due to missing files.")

/usr/local/lib/python3.10/dist-packages/neuralforecast/common/_base_model.py:444: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  content = torch.load(f, **kwargs)
INFO:lightn

Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/neuralforecast/common/_base_model.py:444: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  content = torch.load(f, **kwargs)
INFO:lightn

...

Successfully processed Model AutoBiTCN (bitcn) for Horizon 10.


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Successfully processed Model AutoBiTCN (bitcn) for Horizon 20.


### Merging

In [ ]:
import pandas as pd
import os

class HorizonMerger:
    def __init__(self, base_dir='Data/Test', models=['bitcn', 'tide', 'tft', 'nhits'],
                 model_version='model0_1', horizons=[1], prefixes=[None]):
        """
        Initializes the HorizonMerger with specified parameters.

        Parameters:
        - base_dir (str): Base directory where data is stored.
        - models (list): List of model names to process.
        - model_version (str): Version identifier for the models.
        - horizons (list): List of horizon values to process.
        - prefixes (list): List of prefixes to process, e.g., ['', '8TY'].
        """
        self.base_dir = base_dir
        self.models = models
        self.model_version = model_version
        self.horizons = horizons
        self.prefixes = prefixes  # List of prefixes
        # Mapping from model names to their corresponding column prefixes
        self.model_column_mapping = {
            'bitcn': 'AutoBiTCN',
            'tide': 'AutoTiDE',
            'tft': 'AutoTFT',
            'nhits': 'AutoNHITS'
        }

    def merge_models_for_horizon_and_prefix(self, horizon, prefix):
        """
        Merges CSV files for a specific horizon and prefix, then saves the full and subset data.

        Parameters:
        - horizon (int): The horizon value to process.
        - prefix (str): The prefix to add to the filename.
        """
        # List to store individual DataFrames
        df_list = []

        # Path to the current horizon's directory
        horizon_dir = os.path.join(self.base_dir, f'horizon_{horizon}')

        # Ensure the horizon directory exists
        if not os.path.isdir(horizon_dir):
            print(f"⚠️ Horizon directory does not exist: {horizon_dir}. Creating it...")
            os.makedirs(horizon_dir, exist_ok=True)

        # Read each model's CSV for the current horizon
        for model in self.models:
            # Update file name to include the prefix
            file_name = f'{prefix or ""}{model}_{self.model_version}_horizon_{horizon}.csv'
            file_path = os.path.join(horizon_dir, file_name)
            if not os.path.exists(file_path):
                print(f"❌ File not found: {file_path}")
                return  # Skip processing this horizon if any file is missing
            df = pd.read_csv(file_path)
            df_list.append(df)
            print(f"✅ Loaded {file_path}")

        # Merge all DataFrames on the specified keys
        print(f"🔗 Merging data for horizon {horizon} with prefix '{prefix}'...")
        merged_df = df_list[0]
        for df in df_list[1:]:
            merged_df = pd.merge(merged_df, df, on=['unique_id', 'ds', 'y'], how='inner')
        print("✅ Merging completed.")

        # Define the output filename and path for the full merged data
        full_output_filename = f'{prefix or ""}{"_".join(self.models)}_{self.model_version}_full_horizon_{horizon}.csv'
        full_output_path = os.path.join(horizon_dir, full_output_filename)

        # Save the merged DataFrame
        merged_df.to_csv(full_output_path, index=False)
        print(f"💾 Saved merged data to {full_output_path}")

        # Prepare subset columns based on the mapping
        subset_columns = ['unique_id', 'ds', 'y']
        for model in self.models:
            col_prefix = self.model_column_mapping.get(model)
            if col_prefix:
                subset_columns.extend([col_prefix, f'{col_prefix}1'])
            else:
                print(f"⚠️ No column mapping found for model '{model}'. Skipping its subset columns.")

        # Check for missing columns and handle them
        missing_cols = [col for col in subset_columns if col not in merged_df.columns]
        if missing_cols:
            print(f"⚠️ Missing columns for subset: {missing_cols}")
            for col in missing_cols:
                merged_df[col] = pd.NA

        # Create the subset DataFrame
        df_subset = merged_df[subset_columns].copy()

        # Define the output filename and path for the subset data
        subset_output_filename = f'{prefix or ""}{"_".join(self.models)}_{self.model_version}_sub_horizon_{horizon}.csv'
        subset_output_path = os.path.join(horizon_dir, subset_output_filename)

        # Save the subset DataFrame
        df_subset.to_csv(subset_output_path, index=False)
        print(f"💾 Saved subset data to {subset_output_path}\n")

    def process_all_horizons_and_prefixes(self):
        """
        Processes all specified horizons and prefixes by merging and saving their respective data.
        """
        for prefix in self.prefixes:
            for horizon in self.horizons:
                print(f"🚀 Processing horizon {horizon} with prefix '{prefix}'...")
                self.merge_models_for_horizon_and_prefix(horizon, prefix)
                print(f"✅ Finished processing horizon {horizon} for prefix '{prefix}'\n")

if __name__ == "__main__":
    # Initialize the HorizonMerger with horizons and prefixes
    merger = HorizonMerger(
        horizons=[5, 10, 20],
        prefixes=['AEX']
    )

    # Start processing all specified horizons and prefixes
    merger.process_all_horizons_and_prefixes()
    print("✅ Completed all processing")

🚀 Processing horizon 5 with prefix 'AEX'...
✅ Loaded Data/Test/horizon_5/AEXbitcn_model0_1_horizon_5.csv
✅ Loaded Data/Test/horizon_5/AEXtide_model0_1_horizon_5.csv
✅ Loaded Data/Test/horizon_5/AEXtft_model0_1_horizon_5.csv
✅ Loaded Data/Test/horizon_5/AEXnhits_model0_1_horizon_5.csv
🔗 Merging data for horizon 5 with prefix 'AEX'...
✅ Merging completed.
💾 Saved merged data to Data/Test/horizon_5/AEXbitcn_tide_tft_nhits_model0_1_full_horizon_5.csv
💾 Saved subset data to Data/Test/horizon_5/AEXbitcn_tide_tft_nhits_model0_1_sub_horizon_5.csv

✅ Finished processing horizon 5 for prefix 'AEX'

🚀 Processing horizon 10 with prefix 'AEX'...
✅ Loaded Data/Test/horizon_10/AEXbitcn_model0_1_horizon_10.csv
✅ Loaded Data/Test/horizon_10/AEXtide_model0_1_horizon_10.csv
✅ Loaded Data/Test/horizon_10/AEXtft_model0_1_horizon_10.csv
✅ Loaded Data/Test/horizon_10/AEXnhits_model0_1_horizon_10.csv
🔗 Merging data for horizon 10 with prefix 'AEX'...
✅ Merging completed.
💾 Saved merged data to Data/Test/horiz

### Evaluation

Following the evaluation metrics as well as MCS and DM test from Suoto and Moradi 2024

https://www.emerald.com/insight/content/doi/10.1108/cfri-01-2024-0032/full/html

https://github.com/hugogobato/Can-Transformers-Transform-Financial-Forecasting-

In [ ]:
import os
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import t
from arch.bootstrap import MCS

# Define the horizons and prefixes
horizons = [5, 10, 20]
prefixes = ['AEX']

# Loop over each prefix and horizon
for prefix in prefixes:
    for horizon in horizons:
        print(f"Processing Prefix: '{prefix}', Horizon: {horizon}")

        # Define file paths based on prefix and horizon
        prefix_str = f"{prefix}_" if prefix else ''
        file_name = f"{prefix}bitcn_tide_tft_nhits_model0_1_sub_horizon_{horizon}.csv"
        data_path = f"Data/Test/horizon_{horizon}/{file_name}"
        output_dir = f"Data/Evaluation/horizon_{horizon}"
        os.makedirs(output_dir, exist_ok=True)

        # Read the CSV file
        df = pd.read_csv(data_path)
        df = df.sort_values(by=['unique_id', 'ds']).drop_duplicates(subset=['ds', 'unique_id'])
        df['ds'] = pd.to_datetime(df['ds'])

        # Extract model columns
        model_columns = ['AutoTiDE', 'AutoTiDE1', 'AutoTFT', 'AutoTFT1',
                         'AutoBiTCN', 'AutoBiTCN1', 'AutoNHITS', 'AutoNHITS1']

        # Create a dictionary to hold pivoted DataFrames for each model
        model_dfs = {}
        for model in model_columns:
            model_df = df.pivot(index='ds', columns='unique_id', values=model)
            model_dfs[model] = model_df

        # Pivot actual values
        Actuals = df.pivot(index='ds', columns='unique_id', values='y')

        # Calculate error metrics for each model
        metrics = ['RMSE', 'MAE', 'MAPE', 'QLIKE']
        error_metrics = {metric: {} for metric in metrics}

        for model_name, predictions in model_dfs.items():
            # Align the predictions and actuals
            predictions, actuals = predictions.align(Actuals, join='inner', axis=0)
            errors = predictions - actuals

            # RMSE Calculation
            mse = (errors ** 2).mean().mean()
            error_metrics['RMSE'][model_name] = np.sqrt(mse)

            # MAE Calculation
            mae = errors.abs().mean().mean()
            error_metrics['MAE'][model_name] = mae

            # MAPE Calculation
            with np.errstate(divide='ignore', invalid='ignore'):
                mape = (errors.abs() / actuals).replace([np.inf, -np.inf], np.nan).mean().mean()
            error_metrics['MAPE'][model_name] = mape

            # QLIKE Calculation
            with np.errstate(divide='ignore', invalid='ignore'):
                ratio = actuals / predictions
                qlike = (ratio - np.log(ratio) - 1).replace([np.inf, -np.inf], np.nan).mean().mean()
            error_metrics['QLIKE'][model_name] = qlike

        # Create a DataFrame with error metrics
        error_metrics_df = pd.DataFrame(error_metrics)
        metrics_file = f"{output_dir}/{prefix_str}metrics_horizon_{horizon}.csv"
        error_metrics_df.to_csv(metrics_file)

        # Function to calculate losses for MCS
        def calculate_losses(metric, model_dfs, actuals):
            residuals_dict = {}
            for model_name, predictions in model_dfs.items():
                # Align predictions and actuals
                predictions_aligned, actuals_aligned = predictions.align(actuals, join='inner', axis=0)
                if metric == 'RMSE':
                    residuals = (predictions_aligned - actuals_aligned) ** 2
                elif metric == 'MAE':
                    residuals = (predictions_aligned - actuals_aligned).abs()
                elif metric == 'MAPE':
                    with np.errstate(divide='ignore', invalid='ignore'):
                        residuals = (predictions_aligned - actuals_aligned).abs() / actuals_aligned
                elif metric == 'QLIKE':
                    with np.errstate(divide='ignore', invalid='ignore'):
                        ratio = actuals_aligned / predictions_aligned
                        residuals = ratio - np.log(ratio) - 1
                else:
                    raise ValueError("Unsupported metric")

                residuals = residuals.replace([np.inf, -np.inf], np.nan)
                residuals_dict[model_name] = residuals.mean(axis=1)

            return pd.DataFrame(residuals_dict).dropna()

        # Run MCS procedure for each metric
        pvalues_list = []

        for metric in metrics:
            losses = calculate_losses(metric, model_dfs, Actuals)
            mcs = MCS(losses, size=0.05, method="R", block_size=1000)
            mcs.compute()
            pvalues = mcs.pvalues.reset_index().rename(columns={'Pvalue': f'Pvalue_{metric}'})
            pvalues_list.append(pvalues)

        # Merge p-values and save
        merged_pvalues = pvalues_list[0]
        for pvalues in pvalues_list[1:]:
            merged_pvalues = merged_pvalues.merge(pvalues, on='Model name', how='outer')

        mcs_file = f"{output_dir}/{prefix_str}MCS_horizon_{horizon}.csv"
        merged_pvalues.to_csv(mcs_file, index=False)

        # Function for Diebold-Mariano test
        def dm_test(actual, pred1, pred2, h=1, crit="RMSE"):
            actual, pred1, pred2 = map(np.array, [actual, pred1, pred2])
            T = len(actual)

            # Ensure predictions and actuals are of the same length
            if len(actual) != len(pred1) or len(actual) != len(pred2):
                return np.nan

            # Remove NaN values
            mask = ~np.isnan(actual) & ~np.isnan(pred1) & ~np.isnan(pred2)
            actual = actual[mask]
            pred1 = pred1[mask]
            pred2 = pred2[mask]
            T = len(actual)
            if T == 0:
                return np.nan

            if crit == "RMSE":
                loss_diff = (actual - pred1) ** 2 - (actual - pred2) ** 2
            elif crit == "MAE":
                loss_diff = np.abs(actual - pred1) - np.abs(actual - pred2)
            elif crit == "MAPE":
                with np.errstate(divide='ignore', invalid='ignore'):
                    loss_diff = (np.abs(actual - pred1) / actual) - (np.abs(actual - pred2) / actual)
            elif crit == "QLIKE":
                with np.errstate(divide='ignore', invalid='ignore'):
                    loss1 = actual / np.abs(pred1) - np.log(actual / np.abs(pred1)) - 1
                    loss2 = actual / np.abs(pred2) - np.log(actual / np.abs(pred2)) - 1
                    loss_diff = loss1 - loss2
            else:
                raise ValueError("Unsupported criterion")

            loss_diff = loss_diff[~np.isnan(loss_diff)]
            if len(loss_diff) == 0:
                return np.nan

            d_mean = np.mean(loss_diff)
            gamma = [np.correlate(loss_diff - d_mean, loss_diff - d_mean, 'full')[len(loss_diff)-1:] / len(loss_diff)][0]
            V_d = gamma[0] + 2 * sum(gamma[1:h])
            DM_stat = d_mean / np.sqrt(V_d / len(loss_diff))
            adj = ((len(loss_diff) + 1 - 2 * h + h * (h - 1) / len(loss_diff)) / len(loss_diff)) ** 0.5
            return DM_stat * adj

        # Run DM tests and save results
        models = model_columns
        stocks = Actuals.columns

        for metric in metrics:
            better_count_matrix = pd.DataFrame(0, index=models, columns=models)

            for model_a, model_b in combinations(models, 2):
                counts = {'a_better': 0, 'b_better': 0}
                for stock in stocks:
                    actual = Actuals[stock]
                    pred1 = model_dfs[model_a][stock]
                    pred2 = model_dfs[model_b][stock]
                    combined = pd.concat([actual, pred1, pred2], axis=1).dropna()

                    if combined.empty:
                        continue

                    dm_stat = dm_test(combined.iloc[:, 0], combined.iloc[:, 1], combined.iloc[:, 2], h=1, crit=metric)
                    if np.isnan(dm_stat):
                        continue
                    if dm_stat > 1.96:
                        counts['a_better'] += 1
                    elif dm_stat < -1.96:
                        counts['b_better'] += 1

                better_count_matrix.loc[model_a, model_b] = counts['a_better']
                better_count_matrix.loc[model_b, model_a] = counts['b_better']

            # Summarize results
            better_count_matrix['Outperform Count'] = better_count_matrix.sum(axis=1)
            better_count_matrix.loc['Outperformed Count'] = better_count_matrix.sum(axis=0)
            better_count_matrix.loc['Outperformed Count', 'Outperform Count'] = np.nan
            dm_file = f"{output_dir}/{prefix_str}DM_{metric}_horizon_{horizon}.csv"
            better_count_matrix.to_csv(dm_file)

        print(f"Completed processing for Prefix: '{prefix}', Horizon: {horizon}\n")

Processing Prefix: 'AEX', Horizon: 5
Completed processing for Prefix: 'AEX', Horizon: 5

Processing Prefix: 'AEX', Horizon: 10
Completed processing for Prefix: 'AEX', Horizon: 10

Processing Prefix: 'AEX', Horizon: 20
Completed processing for Prefix: 'AEX', Horizon: 20

